# Exercise 04 - Due Friday, October 21 at 12pm

*Objectives*: Gain experience loading a CSV dataset into a database model you define yourself and using SQL to explore its contents. Explore the data by writing and executing a number of SQL queries using common syntax and functions and describing your findings.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries in the space provided, unless a text answer is requested. Results should be correct, according to the embedded tests. The notebook itself should be completely reproducible; from start to finish, another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Deadline*: Friday, October 21, 12pm. Submit your notebook to Blackboard and push it to your GitHub repository.

## Setup - obtain data and create database

The US City Open Data Census has data on [restaurant inspections from all over the country](http://us-city.census.okfn.org/dataset/food-safety).  Let's take a look at data from Albuquerque.  *Note*: metadata and other details for this dataset are available at http://data.cabq.gov/business/foodinspections/.

Fyi, to save you a step, I've converted this file into a clean CSV file by using the `dos2unix` command to change its line endings and to switch it to use commas instead of tabs using `csvformat -t inputfile.csv > outputfile.csv`.

First we download the dataset:

In [ ]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv

## Problem 1 - examine the data (35 points)

Use `csvstat` and other tools as you see fit and observe its contents.  Review the metadata definitions linked to above.  Describe what you see:  What do all the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some complications or questions you would like to ask?

Insert a combination of code and text/markdown cells here to review the data and describe what you find.

In [ ]:
!mv abq-food-inspections-fy2010.csv food.csv

In [ ]:
!csvcut -n food.csv

In [ ]:
!csvstat food.csv

In [ ]:
##What do all the columns mean?

1: FACILITY_NAME is the name of inspection facility

2: FACILITY_KEY is the unique identification key of the inspection facility

3: SITE_ADDRESS is the stress address of the inspection facility

4: CITY is the city of the inspection facility

5: STATE is the state of the inspection facility

6: ZIP is the zip of the inspection facility

7: OWNER_KEY is the unique identification key of the owner of inspection facility

8: OWNER_NAME is the name of the owner of the inspection facility

9: NATURE_OF_BUSINESS is the type of business

10: STREET_NUMBER is the stress number of the inspection facility

11: STREET_NAME is the stress name of the inspection facility

12: STREET_TYPE is the stress type of the inspection facility

13: POST_DIRECTIONAL is the postical direction of the inspection facility

14: PHONE is the phone of the inspection facility

15: PROGRAM_CATEGORY is the id that indicate the type of facility inspection

16: PROGRAM_CATEGORY_DESCRIPTION is the description of the id that indicate the type of facility inspection

17: INSPECTION_DATE is the date of inspection

18: INSPECTION_TYPE is code for type of inspection

19: INSPECTION_DESC is the description of inspection type

20: SERIAL_NUM is the unique id for the inspection activity

21: ACTION_CODE is the code of taken action

22: ACTION_DESC is the description of take action

23: RESULT_CODE is the code of inspection result

24: RESULT_DESC is the description of inspection result

25: VIOLATION_CODE is the code for the violation

26: VIOLATION_DESC is the escription of the violation

27: INSPECTION_MEMO is the note from the inspection

## Problem 2 - define a database model and load the data (35 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset into the database and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Insert a combination of code and text/markdown cells here to connect to your database, define your table, load its data, and verify the loaded records.

In [ ]:
%load_ext mysql

In [ ]:
!echo 'redspot' | sudo -S service postgresql restart

In [ ]:
!createdb -U dbuser exercise04

In [ ]:
%sql postgresql://dbuser@localhost:5432/exercise04

In [ ]:
%%sql
DROP TABLE IF EXISTS food;
CREATE TABLE food 
(
    FACILITY_NAME VARCHAR(80),
    FACILITY_KEY INTEGER,
    SITE_ADDRESS VARCHAR(40),
    CITY VARCHAR(12),
    STATE CHAR(2),
    ZIP CHAR(10),
    OWNER_KEY INTEGER,
    OWNER_NAME VARCHAR(85),
    NATURE_OF_BUSINESS VARCHAR(25),
    STREET_NUMBER  VARCHAR(7),
    STREET_NAME VARCHAR(24),
    STREET_TYPE VARCHAR(4),
    POST_DIRECTIONAL VARCHAR(3),
    PHONE CHAR(15),
    PROGRAM_CATEGORY VARCHAR(4),
    PROGRAM_CATEGORY_DESCRIPTION VARCHAR(40),
    INSPECTION_DATE TIMESTAMP,
    INSPECTION_TYPE VARCHAR(4),
    INSPECTION_DESC VARCHAR(32),
    SERIAL_NUM VARCHAR(9),
    ACTION_CODE VARCHAR(2),
    ACTION_DESC VARCHAR(40),
    RESULT_CODE VARCHAR(2),
    RESULT_DESC VARCHAR(40),
    VIOLATION_CODE VARCHAR(10),
    VIOLATION_DESC VARCHAR(50),
    INSPECTION_MEMO VARCHAR(50)
)

In [ ]:
%%sql
COPY food FROM '/home/jovyan/work/food.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

In [ ]:
%%sql
SELECT COUNT(*) FROM food;

## Problem 3 - explore your data (30 points)

Now that the data is loaded, ask some questions of it!  Identify key variables of interest and note their ranges along with other useful descriptive statistics.  Choose and define a few lines of inquiry, execute queries for each, and describe what you find in your result sets.  Use any query techniques we've seen in class, including aggregate functions, transformations, subqueries, or others as appropriate. 

If you find interesting patterns, adding plots to your exploration would be useful.

Insert a combination of code and text/markdown cells here to explore your data.

In [ ]:
%%sql
SELECT sub.* FROM
 (
  SELECT facility_name,count(*) As count FROM food
    Group by facility_name
    )sub
ORDER BY count DESC
LIMIT 5

In [ ]:
%%sql
SELECT sub.* FROM
 (
  SELECT zip_address,count(*) As count FROM food
    Group by zip_address
    )sub
ORDER BY count DESC
LIMIT 5

In [ ]:
%%sql

SELECT inspection_des,count(*) as count FROM food
GROUP BY inspection_des
ORDER BY count desc

In [ ]:
%%sql
SELECT action_code FROM food
WHERE inspection_des='EMERGENCY RESPONSE'

In [ ]:
%%sql 
CREATE TABLE T1 AS
(
SELECT zip_address,inspection_date FROM food
WHERE inspection_des='EMERGENCY RESPONSE'
)

In [ ]:
%%sql

SELECT * FROM T1

In [ ]:
%%sql
SELECT sub.* FROM
 (
  SELECT zip_address,count(*) AS count FROM T1
    Group BY zip_address
    )SUB
ORDER BY count DESC
LIMIT 5

In [ ]:
%%sql
SELECT sub.* FROM
 (
  SELECT inspection_date,count(*) As count FROM T1
    Group BY inspection_date
    )sub
ORDER BY count DESC
LIMIT 5

## Bonus (10 points)

We have seen a few simple techniques for normalizing data in class.  Choose a column or set of columns that might be redundant and normalize them out to another table:

* Identify the columns to normalize in the source data table
* Create one or more new tables to hold the normalized records
* Extract data into the new tables
* Re-insert the new foreign key references from the new table into the source table